In [ ]:
# from __future__ import absolute_import

In [ ]:
# default_exp core

# h3_sequence

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 

from h3 import h3

# We will be adding an additional method to the H3 python interface that exists in the underlying C impl
from ctypes import (
    c_int,
    c_void_p,

)

h3.libh3.experimentalH3ToLocalIj.restype = None
h3.libh3.experimentalH3ToLocalIj.argtypes = [h3.H3Index, h3.H3Index, c_void_p]

In [ ]:
#export

def local_ij_delta(origin, target):
    """
    :param origin: H3 string of the origin hexagon
    :param target: H3 string of the target hexagon
    :return: tuple (i, j) of the delta between the two in IJ coordinates
    """
    IJ = c_int * 2
    originIJ = IJ()
    targetIJ = IJ()
    h3.libh3.experimentalH3ToLocalIj(h3.string_to_h3(origin), h3.string_to_h3(origin), originIJ)
    h3.libh3.experimentalH3ToLocalIj(h3.string_to_h3(origin), h3.string_to_h3(target), targetIJ)
    return (targetIJ[0] - originIJ[0], targetIJ[1] - originIJ[1])

In [ ]:
# Verify origin -> origin should be (0, 0)
origin = '89268cd8d57ffff'
ij = local_ij_delta(origin, origin)
assert ij == (0, 0)

target = '89268cd8dcfffff'
ij = local_ij_delta(origin, target)
assert ij == (1, 0)

In [ ]:
from keplergl import KeplerGl 
import pandas as pd
map_1 = KeplerGl(height=500)

df_1 = pd.DataFrame(
    {'hex': ['89268cd8d57ffff', '89268cd8dcfffff']})

map_1.add_data(data=df_1, name='hex_values')
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'hex_values': {'index': [0, 1], 'columns': ['hex'], 'data': [['89268cd8d57ffff'], ['89268cd8dcf…

In [ ]:
#export

def local_ij_delta_to_class(local_ij_delta):
    """
    :param local_ij_delta: tuple (i, j) returned from local_ij_delta
    :return: a value 0-5 for the each of the possible adjecent hexagons, or -1 if
             the (i,j) tuple is representing a non-adjecent hexagon coordinate
    """
    if (local_ij_delta == (0, 1)):
        return 0
    elif (local_ij_delta == (1, 0)):
        return 1
    elif (local_ij_delta == (0, -1)):
        return 2
    elif (local_ij_delta == (-1, 0)):
        return 3
    elif (local_ij_delta == (-1, -1)):
        return 4
    elif (local_ij_delta == (1, 1)):
        return 5
    else:
        return -1

In [ ]:
#export

def coordinatesToH3IndexSequence(coordinates, resolution):
    """
    :param coordinates: array of (lon, lat) tuples
    :param resolution: H3 resolution
    :return: an array of contiguous H3 indices representing the sequence of coordinates.
             there will not be duplicate H3 index value need to each other in the sequence.
    """
    h3_index_sequence = []
    h3_last_index = ""
    for lon_lat in coordinates:
      h3_index = h3.geo_to_h3(lon_lat[1], lon_lat[0], resolution)
      if h3_index != h3_last_index:
        # check to make sure the h3_last_index is a hexagon adjacent to h3_index
        # if not, we use h3_line to fill in the hex indices between them
        line = []
        if (h3_last_index != ""):
            line = h3.h3_line(h3_last_index, h3_index)      
        if (h3_last_index == "" or len(line) == 2):
            h3_last_index = h3_index
            h3_index_sequence.append(h3_index)
        else:
            h3_last_index = h3_index
            h3_index_sequence.extend(line[1:])
    return h3_index_sequence

In [ ]:
sequence = coordinatesToH3IndexSequence([(-105, 40), (-105, 40.1), (-105.1, 40.1)], 9)

from keplergl import KeplerGl 
import pandas as pd
map_2 = KeplerGl(height=500)

df_2 = pd.DataFrame(
    {'hex': sequence})

map_2.add_data(data=df_2, name='hex_values')
map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'hex_values': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

In [ ]:
#export 
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield list(result)
    for elem in it:
        result = result[1:] + (elem,)
        yield list(result)

In [ ]:
#export

def generate_sequences_with_next_hex_class(coordinates, resolution, sequence_length):
    """
    :param coordinates: array of (lon, lat) tuples
    :param resolution: H3 resolution
    :param sequence_length: the length of sequences of h3 indices to return
    :return: iterable of (sequence, hex_class) tuples. The hex_class is representing
             one of the 6 possible hexagons adjacent to the last hexagon in the
             sequence. this represents the next hexagon in the overall sequence
             and can be used as training data.
    """
    overall_sequence = coordinatesToH3IndexSequence(coordinates, resolution)
    
    # generate the sequences with one extra hex index. this last index is the target
    # with which we will use to calculate the next hex class. the last index will get
    # removed before returning the sequence
    sequences = window(overall_sequence, sequence_length + 1)
    for sequence in sequences:
        last_two = sequence[-2:]
        ij_delta = local_ij_delta(last_two[0], last_two[1])
        next_hex_class = local_ij_delta_to_class(ij_delta)
        yield (sequence[:-1], next_hex_class)

    

In [ ]:
#export

import pathlib
from itertools import chain
def generate_sequences_with_next_hex_class_from_path(path_list, resolution, sequence_length, file_to_coordinates):
    """
    :param path: a list of pathlib.PosixPath containing the files to read into lists of coordinates
    :param resolution: H3 resolution
    :param sequence_length: the length of sequences of h3 indices to return
    :param file_to_coordinates: function that takes a file and returns a list of (lon, lat) coordinates
    :return: iterable of (sequence, hex_class) tuples. The hex_class is representing
             one of the 6 possible hexagons adjacent to the last hexagon in the
             sequence. this represents the next hexagon in the overall sequence
             and can be used as training data.
    """
    generators = []
    for file_path in path_list:
        print(file_path)
        coordinates = file_to_coordinates(file_path)
        generators.append(generate_sequences_with_next_hex_class(coordinates, resolution, sequence_length))
        
    return chain.from_iterable(generators)
    
    

In [ ]:
#export 

import fiona
def parse_strava_gpx(file_path):
    layer = fiona.open(file_path, layer='tracks')
    coordinates = layer[0]['geometry']['coordinates'][0]
    layer.close()
    if (coordinates[0][0] > -109 and coordinates[0][0] < -102 and coordinates[0][1] > 37 and coordinates[0][1] < 41):
        return coordinates
    else:
        return []

In [ ]:
path_list = pathlib.Path("/home/jared/Downloads/strava/activities").glob("*.gpx")
sequence_generator = generate_sequences_with_next_hex_class_from_path(path_list, 9, 5, parse_strava_gpx)
list(sequence for sequence, next_hex in sequence_generator)

[['89268138e07ffff',
  '89268138e0fffff',
  '89268138e0bffff',
  '89268138e47ffff',
  '89268138e7bffff'],
 ['89268138e0fffff',
  '89268138e0bffff',
  '89268138e47ffff',
  '89268138e7bffff',
  '89268138e6bffff'],
 ['89268138e0bffff',
  '89268138e47ffff',
  '89268138e7bffff',
  '89268138e6bffff',
  '89268139db7ffff'],
 ['89268138e47ffff',
  '89268138e7bffff',
  '89268138e6bffff',
  '89268139db7ffff',
  '89268138e7bffff'],
 ['89268138e7bffff',
  '89268138e6bffff',
  '89268139db7ffff',
  '89268138e7bffff',
  '89268139db7ffff'],
 ['89268138e6bffff',
  '89268139db7ffff',
  '89268138e7bffff',
  '89268139db7ffff',
  '89268139da3ffff'],
 ['89268139db7ffff',
  '89268138e7bffff',
  '89268139db7ffff',
  '89268139da3ffff',
  '89268139da7ffff'],
 ['89268138e7bffff',
  '89268139db7ffff',
  '89268139da3ffff',
  '89268139da7ffff',
  '89268139dafffff'],
 ['89268139db7ffff',
  '89268139da3ffff',
  '89268139da7ffff',
  '89268139dafffff',
  '89268139d37ffff'],
 ['89268139da3ffff',
  '89268139da7ffff',
  '8